In [1]:
import random

# Ejercicio N°1
Se tiene un RDD con el registro de notas de los alumnos de la forma (padrón, materia, nota,
fecha). Se pide resolver utilizando PySpark:

- A. Cuántos alumnos aprobaron al menos 1 materia en los últimos 2 años.
- B. Un RDD conteniendo el promedio de notas de cada alumno de la forma (padrón,
promedio).
- C. El nombre y apellido del alumno con mejor promedio. Para esto puede utilizarse un
segundo RDD alumnos con registros (padron, nombre y apellido).


In [2]:
notas = []
padrones = []
cant_alumnos = 50
for i in range(cant_alumnos):
    padron = random.randint(99900,100000)
    while(padron in padrones): padron = random.randint(99900,100000)
    padrones.append(padron)
    materia = random.choice(["AM3","AM2","TALLER","LABORATORIO","DATOS"])
    nota = random.randint(1,10)
    fecha = "{}-{}-{}".format(random.choice(['2017','2018','2019']),
                                        random.randint(1,12),
                                        random.randint(1,28))
    tupla = (padron,materia,nota,fecha)
    notas.append(tupla)


In [3]:
notasRDD = sc.parallelize(notas)

In [4]:
#A
notasRDD.filter(lambda x: x[3] >= '2018-01-01' and x[3]<= '2019-12-31' and x[2]>=4)\
        .count()  

14

In [5]:
#B
promediosRDD = notasRDD.map(lambda x: (x[0], (x[2],1)))\
                       .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                       .map(lambda x: (x[0], x[1][0]/x[1][1]))

In [6]:
alumnos = []
for i in range(cant_alumnos):
    padron = padrones[i]
    nombre = random.choice(["Gustavo","Nicolas","Mariana","Florencia","Ayelen","Pedro","Gonzalo"])
    apellido = random.choice(["Alvarez","Fernandez","Gonzalez","Ramirez","Rodriguez"])
    alumno = (padron, nombre, apellido)
    alumnos.append(alumno)

In [7]:
alumnosRDD = sc.parallelize(alumnos)

In [8]:
#C
alumnosRDD.map(lambda x: (x[0], (x[1] + ' ' + x[2])))\
          .join(promediosRDD)\
          .map(lambda x: (x[1][0], x[1][1]))\
          .reduce(lambda x,y: x if x[1]>=y[1] else y)

('Gonzalo Alvarez', 10.0)

# Ejercicio N°2
Se tiene un RDD registros de ventas de producto con la forma (fecha de venta, código de
producto, precio de venta) y en otro RDD detalle de los productos con (código de producto,
descripción del producto, categoría). Se pide resolver utilizando PySpark:
- A. Cuál es el producto más vendido.
- B. Cuál es la categoría de productos más vendida.
- C. Cuál es el top5 de productos más vendidos generando un RDD con (código de
producto, descripción, cantidad de ventas)
- D. Cuál es el producto que registró mayor aumento de precio en el último año, tomando
para este análisis solo los productos que cuenten con al menos 50 ventas en el último
año.
- E. Idem anterior, pero calculando la categoría de productos que registró mayor variación
de precios en el último año.

In [101]:
ventas = []
productos = []

for i in range(50):
    fecha_de_venta = "{}-{}-{}".format(random.choice(["2018","2019"])
                                                     ,random.randint(1,12)
                                                     ,random.randint(1,28))
    codigo_de_producto = random.randint(1,10)
    precio_de_venta = codigo_de_producto * 25.2
    venta = (fecha_de_venta, codigo_de_producto, precio_de_venta)
    ventas.append(venta)
for i in range(1,10):
    codigo_de_producto = i
    descripcion = "descripcion" + ' ' + str(i)
    categoria = random.choice(["Limpieza","Producto de cocina", "Vestimenta", "Juguete"])
    producto = (codigo_de_producto, descripcion, categoria)
    productos.append(producto)

In [102]:
ventasRDD = sc.parallelize(ventas)
productosRDD = sc.parallelize(productos)

In [103]:
#A - (Codigo_producto, cantidad_vendida)
ventasRDD.map(lambda x: (x[1],1))\
         .reduceByKey(lambda x,y: x+y)\
         .reduce(lambda x,y: x if x[1]>y[1] else y)

(9, 9)

In [104]:
#B
cant = ventasRDD.map(lambda x: (x[1],1))\
                .reduceByKey(lambda x,y: x+y)
#cant es un RDD con las cantidades de venta de cada prod [(id_producto, cant)]
productosRDD.map(lambda x: (x[0], x[2]))\
            .join(cant)\
            .map(lambda x: (x[1][0], x[1][1]))\
            .reduce(lambda x,y: x if x[1] >= y[1] else y ) 
#Me devuelve la tupla (categoria,cant), puedo hacer [0] y obtener la categoria

('Juguete', 9)

In [105]:
#C
productosRDD_map = productosRDD.map(lambda x: (x[0], x[1]))
top5RDD = sc.parallelize(cant.top(5,key = lambda x: x[1]))\
            .join(productosRDD_map)\
            .map(lambda x: (x[0],x[1][1],x[1][0]))
top5RDD.count()

5

In [108]:
top5RDD.takeOrdered(5,key = lambda x: -x[2])

[(9, 'descripcion 9', 9),
 (3, 'descripcion 3', 6),
 (7, 'descripcion 7', 6),
 (8, 'descripcion 8', 5),
 (6, 'descripcion 6', 5)]